In [1]:
import trading_vix_and_spy

In [2]:
local_env = trading_vix_and_spy.trading_vix_and_spy()

In [3]:
local_env.reset()

array([0.1385490996398558, 0.1290820882088209, 0.1095566639165979,
       0.1385490996398558, 0.1385490996398558, 0.1385490996398558,
       0.0602439960444004, 0.0910825891099319, 0.092747308269233,
       0.21101983, -0.2915927971188475, -0.1376122412241224,
       -0.1501286282157054, -0.2915927971188475, -0.2915927971188475,
       -0.2915927971188475, -0.0283591328792542, -0.0431152157200982,
       -0.063465268261073, 0.96337626, 0.1064252100840338,
       0.0515103810381037, 0.1459730405760143, 0.1064252100840338,
       0.1064252100840338, 0.1064252100840338, 0.1238114445716063,
       0.1203029686123038, 0.1168444428977716, 0.28675711,
       0.0351536614645857, 0.1204903690369037, 0.0633702130053251,
       0.0351536614645857, 0.0351536614645857, 0.0351536614645857,
       0.0291373726374737, 0.0721682429265182, 0.0710380708722835,
       0.23556182, 26.02, -0.0959875150060024, -0.0457041104110411,
       -0.0553239555988899, -0.0959875150060024, -0.0959875150060024,
       -

In [4]:
#parameters for spy
spy_parameters = {}
spy_parameters['inner_denoise_std'] = 2
spy_parameters['outer_denoise_std'] = 2
spy_parameters['buy_1_threshold'] = 1
spy_parameters['buy_2_threshold'] = 2
spy_parameters['sell_1_threshold'] = 1
spy_parameters['sell_2_threshold'] = 2
spy_parameters['inner_window_length_interval'] = 10 #in unit of days
spy_parameters['outer_window_length_interval'] = 25
spy_parameters['buy_1_wait'] = 6 #in unit of days
spy_parameters['buy_2_wait'] = 6
spy_parameters['sell_1_wait'] = 1
spy_parameters['sell_2_wait'] = 1
spy_parameters['min_profit'] = 0.5 #in unit of percentage
spy_parameters['pct_portfolio_in_vix_model'] = 0.2 #in unit of percentage
spy_parameters['max_spy_positions'] = 8

#convert the window length from unit of days into interval count
days_into_interval = 7

In [5]:
import numpy as np
import pandas as pd
import trading_vix_and_spy_utils
import ast

In [9]:
#load the vix model
import pickle
with open('lifeline_model.pickle', 'rb') as f:
    vix_model = pickle.load(f)
last_valid_data_for_vix = 52

In [10]:
local_env.seed(2)
full_observation = local_env.reset()

prediction = vix_model.predict_survival_function(full_observation[0:last_valid_data_for_vix].reshape((1,-1)))
np.mean(prediction[0].tolist()[-10:])

0.34118974143724723

In [ ]:
current_spy_stock_value = 0
current_spy_cash_value = local_env.spy_cash
inner_lower_bound_history = []
inner_upper_bound_history = []
outer_lower_bound_history = []
outer_upper_bound_history = []
spy_buy_sell = []
spy_price_history = None
total_portfolio_value_history = None
cash_history = []
ratio_in_stock_form = []



#other things related to spy trading
last_buy_1_time = -99999
last_buy_2_time = -99999
last_sell_1_time = -99999
last_sell_2_time = -99999
min_spy_purchase_price = float('inf')
time_index = 0

In [ ]:
local_env.seed(2)
full_observation = local_env.reset()
spy_observation = ast.literal_eval(full_observation[-2])
ratio_in_stock_form.append(full_observation[-1])

while True:

    inner_lower_bound, inner_upper_bound = trading_vix_and_spy_utils.generate_spy_bounds(
        spy_observation,
        int(spy_parameters['inner_window_length_interval']*days_into_interval),
        spy_parameters['inner_denoise_std'],
        spy_parameters['buy_1_threshold'],
        spy_parameters['sell_1_threshold']
    )

    outer_lower_bound, outer_upper_bound = trading_vix_and_spy_utils.generate_spy_bounds(
        spy_observation,
        int(spy_parameters['outer_window_length_interval']*days_into_interval),
        spy_parameters['outer_denoise_std'],
        spy_parameters['buy_2_threshold'],
        spy_parameters['sell_2_threshold']
    )
    
    current_spy_price = spy_observation[-1]
    
    spy_action = 0.0 #do nothing spy action is 0.0
    spy_action_type = None
    #see if we can buy
    if current_spy_price < inner_lower_bound and current_spy_price > outer_lower_bound:
        time_since_last_buy = (time_index-last_buy_1_time) #time_since_last_buy is in unit of interval
        if time_since_last_buy > spy_parameters['buy_1_wait']*days_into_interval:
            spy_action = 1
            spy_action_type = 'buy_1'
    if current_spy_price < inner_lower_bound and current_spy_price < outer_lower_bound:
        time_since_last_buy = (time_index-last_buy_2_time)
        if time_since_last_buy > spy_parameters['buy_2_wait']*days_into_interval:
            spy_action = 1
            spy_action_type = 'buy_2'
    
    #see if we can sell
    if current_spy_price > inner_upper_bound and current_spy_price < outer_upper_bound:
        time_since_last_sell = (time_index-last_sell_1_time)
        if time_since_last_sell > spy_parameters['sell_1_wait']*days_into_interval:
            if current_spy_price > min_spy_purchase_price*(1+spy_parameters['min_profit']/100):
                spy_action = -1
                spy_action_type = 'sell_1'
            
    if current_spy_price > inner_upper_bound and current_spy_price > outer_upper_bound:
        time_since_last_sell = (time_index-last_sell_2_time)
        if time_since_last_sell > spy_parameters['sell_2_wait']*days_into_interval:
            if current_spy_price > min_spy_purchase_price*(1+spy_parameters['min_profit']/100):
                spy_action = -1
                spy_action_type = 'sell_2'
            
    #compute the max allowed buy dollar value
    prediction = vix_model.predict_survival_function(full_observation[0:last_valid_data_for_vix].reshape((1,-1)))
    vix_model_prediction = np.mean(prediction[0].tolist()[-10:])
    spy_buy_cap = (current_spy_stock_value+current_spy_cash_value)*(1-spy_parameters['pct_portfolio_in_vix_model']) + \
                  (current_spy_stock_value+current_spy_cash_value)*spy_parameters['pct_portfolio_in_vix_model']*vix_model_prediction
                
    #form the joint action
    action_df = pd.DataFrame()
    action_df['0'] = [spy_action]
    action_df['1'] = [spy_parameters['max_spy_positions']] 
    action_df['2'] = [spy_buy_cap]
    
    action_array = action_df.iloc[0].tolist()
    action_array = np.reshape(action_array,(-1,))
    
    #apply action
    full_observation, reward, done, info = local_env.step(action_array)
    if done != True:
        spy_observation = ast.literal_eval(full_observation[-2])
        ratio_in_stock_form.append(full_observation[-1])
        
    cash_history.append(info['spy_cash'])

    
    #prepare for the next round of action
    if info['bought_spy']:
        if spy_action_type == 'buy_1':
            last_buy_1_time = time_index
        if spy_action_type == 'buy_2':
            last_buy_2_time = time_index
    if info['sold_spy']:
        if spy_action_type == 'sell_1':
            last_sell_1_time = time_index
        if spy_action_type == 'sell_2':
            last_sell_2_time = time_index
            
    min_spy_purchase_price = info['min_spy_purchase_price']
    time_index += 1
    
    #record information for plotting
    inner_lower_bound_history.append(inner_lower_bound)
    inner_upper_bound_history.append(inner_upper_bound)
    outer_lower_bound_history.append(outer_lower_bound)
    outer_upper_bound_history.append(outer_upper_bound)
    
    if info['bought_spy']:
        spy_buy_sell.append(1)
    elif info['sold_spy']:
        spy_buy_sell.append(-1)
    else:
        spy_buy_sell.append(0)

    current_spy_cash_value = local_env.spy_cash
    current_spy_stock_value = trading_vix_and_spy_utils.total_value_in_spy_stock(current_spy_cash_value,info['current_spy_positions'],current_spy_price)
    
    if done:
        total_portfolio_value_history = info['total_portfolio_value']
        spy_price_history = info['spy_price_trajectory']
        break
    

In [ ]:
print(len(inner_lower_bound_history))
print(len(spy_price_history))
print(len(spy_buy_sell))
print(len(ratio_in_stock_form))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (15,8)

already_plotted_sell_legend = False
already_plotted_buy_legend = False

plt.plot(spy_price_history,label = 'stock price')
plt.plot(inner_lower_bound_history,label = 'inner lower bound')
plt.plot(outer_lower_bound_history,label = 'outer lower bound')
plt.plot(inner_upper_bound_history,label = 'inner upper bound')
plt.plot(outer_upper_bound_history,label = 'outer upper bound')

for time_index in range(0,len(spy_price_history)):

    if spy_buy_sell[time_index]==1:
        #print('the buy price is',price_history[time_index])
        if already_plotted_sell_legend == False:
            plt.scatter(time_index,spy_price_history[time_index],color = 'b',label = 'buy')
            already_plotted_sell_legend = True
        else:
            plt.scatter(time_index,spy_price_history[time_index],color = 'b')

    elif spy_buy_sell[time_index]==-1:
        #print('the sell price is',price_history[time_index])
        #print(' ')
        if already_plotted_buy_legend == False:
            plt.scatter(time_index,spy_price_history[time_index],color = 'r',label = 'sell')
            already_plotted_buy_legend = True
        else:
            plt.scatter(time_index,spy_price_history[time_index],color = 'r')
            
plt.legend()
plt.show()



In [ ]:
plt.rcParams["figure.figsize"] = (15,8)

fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('Hour')
ax1.set_ylabel('remaining cash', color=color)
ax1.plot(cash_history, color=color)
ax1.tick_params(axis='y', labelcolor=color)
#ax1.hlines(6,0,len(plot_prediction),colors = 'black')


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('spy price', color=color)  # we already handled the x-label with ax1
ax2.plot(spy_price_history, color=color)
ax2.tick_params(axis='y', labelcolor=color)
fig.tight_layout()  # otherwise the right y-label is slightly clipped

#ax2.hlines(0.5,0,len(plot_prediction),colors = 'black')


plt.show()   

In [ ]:
plt.plot(total_portfolio_value_history)
plt.show()

In [ ]:
local_env.total_realized_gain

In [ ]:
plt.rcParams["figure.figsize"] = (15,8)

fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('Hour')
ax1.set_ylabel('ratio_in_stock_form', color=color)
ax1.plot(ratio_in_stock_form, color=color)
ax1.tick_params(axis='y', labelcolor=color)
#ax1.hlines(6,0,len(plot_prediction),colors = 'black')


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('spy price', color=color)  # we already handled the x-label with ax1
ax2.plot(spy_price_history, color=color)
ax2.tick_params(axis='y', labelcolor=color)
fig.tight_layout()  # otherwise the right y-label is slightly clipped

#ax2.hlines(0.5,0,len(plot_prediction),colors = 'black')


plt.show()   

In [ ]:
total_portfolio_return_history = np.asarray(total_portfolio_value_history)/total_portfolio_value_history[0]
sharp_ratio = total_portfolio_return_history[-1] - total_portfolio_return_history[0]
sharp_ratio = sharp_ratio / np.std(total_portfolio_return_history) 
sharp_ratio

In [ ]:
sharp_ratio = total_portfolio_value_history[-1]/total_portfolio_value_history[0]-1
sharp_ratio = sharp_ratio / np.std(total_portfolio_value_history) 
sharp_ratio

In [ ]:
np.std(total_portfolio_value_history) 